In [ ]:
import os
import cv2
import torch
import random
import numpy as np 
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Define Constant

In [ ]:
FILE_PATH= "/kaggle/input/shop-lifter/Shop DataSet/non shop lifters/shop_lifter_n_0.mp4" # Example to test functions
video_path = '/kaggle/input/shop-lifter/Shop DataSet'  # Path to your dataset
class_0 = "non shop lifters"
class_1 = "shop lifters"
out_path = "/kaggle/working/aug_vid"
if not os.path.exists(out_path):
    os.makedirs(out_path)
# Parameters
             # Number of frames to extract from each video
height, width = 90, 90        # Dimensions to resize each frame
channels = 3                  # Number of channels (RGB)
batch_size = 4          # Batch size for training
epochs = 30                   # Number of training epochs

### List all Videos with labels 

In [ ]:
all_videos = []
for label, class_name in enumerate(os.listdir(video_path)):
    class_path = os.path.join(video_path, class_name)
    if os.path.isdir(class_path):
        for video_name in os.listdir(class_path):
            video_file = os.path.join(class_path, video_name)
            if video_file.endswith('.mp4'):  # Adjust for your video format
                all_videos.append((video_file, label))


print(f"Total videos: {len(all_videos)}")


In [ ]:
all_videos [0]

# Video Augmentation

In [ ]:
import imgaug.augmenters as iaa

In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import random

def augment_video(frames):
    """Apply a random combination of augmentations to video frames using albumentations."""
    
    # Define the individual augmentations
    transform = A.Compose([
        A.HorizontalFlip(p=0.5),  # Flip with a 50% chance
        A.Rotate(limit=10, p=0.5),  # Randomly rotate between -10 and 10 degrees
        A.RandomCrop(width=20, height=20, p=0.5),  # Random crop
        A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),  # Random brightness/contrast adjustment
    ])

    # Apply augmentations to each frame
    frames_aug = [transform(image=frame)['image'] for frame in frames]

    return frames_aug


In [ ]:
# Split into training and testing sets
train_videos, test_videos = train_test_split(all_videos, test_size=0.2, stratify=[label for _, label in all_videos], random_state=42)

print(f"Total videos: {len(all_videos)}")
print(f"Training videos: {len(train_videos)}")
print(f"Testing videos: {len(test_videos)}")

In [ ]:
def count_frames(video_path):
    """Count the number of frames in a video file."""
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error opening video file {video_path}")
        return 0
    
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    return frame_count

In [ ]:
def average_frames(video_dir):
    """Calculate the average number of frames for all videos in the specified directory."""
    video_files = [f for f in os.listdir(video_dir) if f.endswith('.mp4')]
    total_frames = 0
    video_count = 0
    
    for video_file in video_files:
        video_path = os.path.join(video_dir, video_file)
        frame_count = count_frames(video_path)
        if frame_count > 0:  # Ensure valid frame count
            total_frames += frame_count
            video_count += 1

    # Calculate average
    average = total_frames / video_count if video_count > 0 else 0
    return average


In [ ]:
# Path to your dataset
avg_frames_0 = average_frames(os.path.join(video_path,class_0))
print(f"Average number of frames per video in class 0: {avg_frames_0:.2f}")

In [ ]:
# Path to your dataset
avg_frames_1 = average_frames(os.path.join(video_path,class_1))
print(f"Average number of frames per video in class 1: {avg_frames_1:.2f}")

In [ ]:
Total_frames_average  = (avg_frames_0  + avg_frames_1) //2
Total_frames_average

In [ ]:
num_frames = int(Total_frames_average * .8 )
# num_frames = int(Total_frames_average)

In [ ]:
class_names = []
video_counts = []
for label, class_name in enumerate(os.listdir(video_path)):
    class_path = os.path.join(video_path, class_name)
    if os.path.isdir(class_path):  # Ensure it's a directory (class folder)
        class_names.append(class_name)
        video_counts.append(len(os.listdir(class_path)))
        

In [ ]:
label = []
for i, loops in enumerate(video_counts):
    for j in range(loops):
        label.append(i)
print(len(label))    

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(label),
    y=label
)

# Convert to a dictionary
class_weight_dict = dict(enumerate(class_weights))
print(class_weight_dict) 

In [ ]:
import datetime
def process_and_augment_videos(all_videos, out_path, num_frames=200, frame_step=15):
    for i in range(0, len(all_videos), frame_step):
        video, label = all_videos[i]
        
        if label == 1:
            frames = []
            cap = cv2.VideoCapture(video)
            
            if not cap.isOpened():
                print(f"Error opening video file {video}")
                continue

            frame_count = 0
            # Read frames from the video
            while cap.isOpened() and frame_count < num_frames:
                ret, frame = cap.read()
                if not ret:
                    break

                # Resize frame to fixed dimensions if needed
                frame = cv2.resize(frame, (width, height))
                
                # Append frame to the list
                frames.append(frame)
                frame_count += 1

            cap.release()

            # Convert list of frames to NumPy array for augmentation
            video_np = np.array(frames)
            
            # Perform augmentation
            augmented_frames = augment_video(video_np)

            # Save augmented video
            video_filename = f'{str(datetime.datetime.now())}.mp4'
            video_output_path = os.path.join(out_path, video_filename)

            # Define video writer with the same width and height as frames
            out = cv2.VideoWriter(video_output_path, cv2.VideoWriter_fourcc(*'mp4v'), 20, (width, height))

            # Write augmented frames to the output video file
            for frame in augmented_frames:
                out.write(frame)

            out.release()

            # Append new video path and label to `all_videos`
            all_videos.append((video_output_path, label))

        

In [ ]:
process_and_augment_videos(all_videos, out_path, num_frames=num_frames, frame_step=2)

In [ ]:
print(f"Total videos: {len(all_videos)}")

In [ ]:
# Split into training and testing sets
train_videos, test_videos = train_test_split(all_videos, test_size=0.2, stratify=[label for _, label in all_videos], random_state=42)

print(f"Total videos: {len(all_videos)}")
print(f"Training videos: {len(train_videos)}")
print(f"Testing videos: {len(test_videos)}")


# Video generator

In [ ]:
def video_generator(videos, batch_size, num_frames, height, width, channels, class_weights ):
    """Generator function to yield video batches with class weights from pre-defined video list."""
    while True:  # Loop indefinitely
        random.shuffle(videos)  # Shuffle the videos for each epoch

        data = []
        labels = []
        weights = []

        # Iterate through the shuffled list of videos
        for video_file, label in videos:
            frames = []
            cap = cv2.VideoCapture(video_file)

            if not cap.isOpened():
                print(f"Error opening video file {video_file}")
                continue

            frame_count = 0
            # Read frames from the video
            while cap.isOpened() and frame_count < num_frames:
                ret, frame = cap.read()
                if not ret:
                    break
                # Resize frame and append to the frames list
                frame = cv2.resize(frame, (width, height))
                frame = frame.astype('float32') / 255.0
                frames.append(frame)
                frame_count += 1
            
            cap.release()

            # If the video has fewer frames than required, pad with zeros
            while len(frames) < num_frames:
                frames.append(np.zeros((height, width, channels), dtype=np.uint8))

            # Convert frames list to numpy array
            frames = np.array(frames)
          
            
            # Append the processed video, its label, and the corresponding weight
            data.append(frames)
            labels.append(label)
            
            
           
            #  weights.append(class_weights[label])  # Use class_weights

            # Yield batch of videos when data list reaches batch_size
            if len(data) == batch_size:
#                 yield np.array(data), np.array(labels), np.array(weights)
                yield np.array(data), np.array(labels)
                data = []  # Reset data list for the next batch
                labels = []  # Reset labels list for the next batch
#                 weights = []  # Reset weights list for the next batch

        # If there are leftover videos that don't fill a full batch, yield them
        if len(data) > 0:
#             yield np.array(data), np.array(labels), np.array(weights)
            yield np.array(data), np.array(labels)


In [ ]:
# Create the generators
gen = video_generator(train_videos, batch_size, num_frames, height, width, channels, class_weights )
test_gen = video_generator(test_videos, batch_size, num_frames, height, width, channels, class_weights)


In [ ]:
for batch in gen:
    videos , labels = batch
    print(videos.shape)
    print(labels)

    break

# Viualization

In [ ]:
def plot_images(video_list):
    """Plot a list of 4 images in a 2x2 grid."""
    fig, axes = plt.subplots(2,4, figsize=(12, 6))  
    axes = axes.flatten()  
    
    for i, video in enumerate(video_list):
        axes[i].imshow(video[20,:,:,:] )
        axes[i].axis('off')  

    plt.tight_layout()
    plt.show()


plot_images(videos)



In [ ]:
# Plotting the results
plt.figure(figsize=(10, 6))
plt.bar(class_names, video_counts, color='skyblue')
plt.xlabel('Class Name')
plt.ylabel('Number of Videos')
plt.title('Number of Videos per Class')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
all_videos[0]

In [ ]:
# Count the total number of videos in the list
total_videos = len(all_videos)
print("Total number of videos:", total_videos)

# Count the number of videos for each class
class_0_count = sum(1 for video in all_videos if video[1] == 0)
class_1_count = sum(1 for video in all_videos if video[1] == 1)

print("Number of class 0 videos:", class_0_count)
print("Number of class 1 videos:", class_1_count)

In [ ]:
# Count the number of videos for each class
class_0_count = sum(1 for video in all_videos if video[1] == 0)
class_1_count = sum(1 for video in all_videos if video[1] == 1)


classes = ['Non Shop Lifters (Class 0)', 'Shop Lifters (Class 1)']
counts = [class_0_count, class_1_count]



plt.figure(figsize=(10, 6))
plt.bar(classes, counts, color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Number of Videos')
plt.title('Distribution of Videos by Class After Aug')
plt.xticks(rotation=10)
plt.grid(axis='y')
plt.show()

# Pretrained model

In [ ]:
import tensorflow as tf

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:
from tensorflow.keras import backend as K

# Clear the session to release memory
K.clear_session()

In [ ]:
steps_per_epoch = len(train_videos) // batch_size
validation_steps = len(test_videos) // batch_size

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,ReduceLROnPlateau,TensorBoard
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
checkpoint = ModelCheckpoint('new_ShopLifter.keras',
    monitor='val_loss', 
    mode='min',
    save_best_only=True,
    save_freq='epoch',
    verbose=1)

In [ ]:
tf.get_logger().setLevel('ERROR')  
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
tf.get_logger().setLevel('INFO')  # Set to 'DEBUG' for more details


In [ ]:
# Load the pretrained MobileNetV2 model
base_model = tf.keras.applications.MobileNetV2(
    include_top=False, 
    weights='imagenet', 
    input_shape=(height, width, channels)
)

# Freeze the base model layers
base_model.trainable = False

# Add your custom layers on top of the base model
model = tf.keras.Sequential([
    tf.keras.layers.TimeDistributed(base_model, input_shape=(num_frames, height, width, channels)),
    tf.keras.layers.GlobalAveragePooling3D(),
    tf.keras.layers.Dense(2, activation='softmax')  # Use softmax for binary classification
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
try:
    hist = model.fit(
        gen,
        steps_per_epoch=steps_per_epoch,
        epochs=epochs,
        validation_data=test_gen,
        validation_steps=validation_steps,
        callbacks=[early_stopping, checkpoint, lr_scheduler],
        verbose=1  # Verbose output to monitor training progress
    )
except Exception as e:
    print(f"Training error: {e}")

# Evaluatation

In [ ]:
# Plot the accuracy
plt.plot(hist.history['accuracy'], label='Training Accuracy')
plt.plot(hist.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Plot the loss
plt.plot(hist.history['loss'], label='Training Loss')
plt.plot(hist.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
# plt.ylim(0, 10)
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Save the model
model.save('shop_lifter_model.h5')

In [ ]:
test_gen = video_generator(test_videos, batch_size, num_frames, height, width, channels, class_weights)


In [ ]:
model = tf.keras.models.load_model("shop_lifter_model.h5")

In [ ]:
loss, accuracy = model.evaluate(test_gen, steps=validation_steps)
print(f"Loss: {loss}")
print(f"Accuracy: {accuracy}")

In [ ]:
# Get the predicted class probabilities
y_pred = model.predict(test_gen, steps=validation_steps)

# Convert probabilities to class predictions (assuming binary classification)
y_pred_classes = np.argmax(y_pred, axis=1)



In [ ]:
test_videos[0]

In [ ]:
# Function to load video frames
test_gen = video_generator(test_videos, batch_size, num_frames, height, width, channels, class_weights)

# Parameters
num_frames = num_frames  
target_size = (width, height)  

# Prepare lists for true and predicted labels
results = []

# Loop through the test videos to evaluate
for video_frames, true_label in test_gen:
    if len(results) > 100:
        break
    if len(video_frames) == 0:
        continue  

    # Make predictions
    predictions = model.predict(video_frames)
    predicted_label = np.argmax(predictions, axis=1)  # Get the predicted classes for the batch

    # Store the true and predicted labels for each video in the batch
    for i in range(len(true_label)):
        results.append((true_label[i], predicted_label[i]))

# Convert results to a NumPy array for confusion matrix
results = np.array(results)

# Extract true and predicted labels separately
true_labels = results[:, 0]
predicted_labels = results[:, 1]




In [ ]:
predicted_labels.shape

In [ ]:
true_labels.shape

In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
# Generate confusion matrix
# Optionally, generate a confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(6,6))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Non Shoplifters', 'Shoplifters'], yticklabels=['Non Shoplifters', 'Shoplifters'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Generate classification report
report = classification_report(true_labels, predicted_labels, target_names=['Non Shoplifters', 'Shoplifters'])
print(report)
